In [31]:
import scanpy as sc
import torch
import anndata as ad
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [2]:
# data_full = 'data/GSE194122_openproblems_neurips2021_cite_BMMC_processed.h5ad'
# # Load the dataset into Scanpy using the backup_url argument
# adata_full = sc.read_h5ad(data_full)
# adata_full

In [3]:
data_small = "data/pbmc_10k_protein_v3_raw_feature_bc_matrix.h5"
adata_small = sc.read_10x_h5(data_small, genome=None, gex_only=False, backup_url=None)

/home/dennis/miniconda3/envs/compbio/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [5]:
adata_small.var_names_make_unique()
adata_small.layers["counts"] = adata_small.X.copy()
sc.pp.filter_genes(adata_small, min_counts=10) # number of times that RNA is present in the dataset
sc.pp.filter_cells(adata_small, min_counts=100) # number of rna molecules in each cell
adata_small

AnnData object with n_obs × n_vars = 80773 × 15989
    obs: 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'pattern', 'read', 'sequence', 'n_counts'
    layers: 'counts'

In [6]:
adata_small.var["feature_types"].value_counts()
# The number of unique genes/antibodies in the dataset.

feature_types
Gene Expression     15972
Antibody Capture       17
Name: count, dtype: int64

In [7]:
protein = adata_small[:, adata_small.var["feature_types"] == "Antibody Capture"].copy()
rna = adata_small[:, adata_small.var["feature_types"] == "Gene Expression"].copy()

In [8]:
protein.var

,gene_ids,feature_types,genome,pattern,read,sequence,n_counts
CD3_TotalSeqB,CD3,Antibody Capture,,^NNNNNNNNNN(BC)NNNNNNNNN,R2,AACAAGACCCTTGAG,10826011.0
CD4_TotalSeqB,CD4,Antibody Capture,,^NNNNNNNNNN(BC)NNNNNNNNN,R2,TACCCGTAATAGCGT,10043849.0
CD8a_TotalSeqB,CD8a,Antibody Capture,,^NNNNNNNNNN(BC)NNNNNNNNN,R2,ATTGGCACTCAGATG,8009552.0
CD14_TotalSeqB,CD14,Antibody Capture,,^NNNNNNNNNN(BC)NNNNNNNNN,R2,GAAAGTCAAAGCACT,4112917.0
CD15_TotalSeqB,CD15,Antibody Capture,,^NNNNNNNNNN(BC)NNNNNNNNN,R2,ACGAATCAATCTGTG,8797653.0
CD16_TotalSeqB,CD16,Antibody Capture,,^NNNNNNNNNN(BC)NNNNNNNNN,R2,GTCTTTGTCAGTGCA,9301829.0
CD56_TotalSeqB,CD56,Antibody Capture,,^NNNNNNNNNN(BC)NNNNNNNNN,R2,GTTGTCCGACAATAC,1463992.0
CD19_TotalSeqB,CD19,Antibody Capture,,^NNNNNNNNNN(BC)NNNNNNNNN,R2,TCAACGCTTGGCTAG,730998.0
CD25_TotalSeqB,CD25,Antibody Capture,,^NNNNNNNNNN(BC)NNNNNNNNN,R2,GTGCATTCAACAGTA,743054.0
CD45RA_TotalSeqB,CD45RA,Antibody Capture,,^NNNNNNNNNN(BC)NNNNNNNNN,R2,GATGAGAACAGGTTT,15976956.0


In [9]:
rna.var

,gene_ids,feature_types,genome,pattern,read,sequence,n_counts
AL627309.1,ENSG00000238009,Gene Expression,GRCh38,,,,14.0
AL669831.5,ENSG00000237491,Gene Expression,GRCh38,,,,509.0
LINC00115,ENSG00000225880,Gene Expression,GRCh38,,,,197.0
FAM41C,ENSG00000230368,Gene Expression,GRCh38,,,,330.0
NOC2L,ENSG00000188976,Gene Expression,GRCh38,,,,2037.0
...,...,...,...,...,...,...,...
AC007325.4,ENSG00000278817,Gene Expression,GRCh38,,,,61.0
AL354822.1,ENSG00000278384,Gene Expression,GRCh38,,,,31.0
AC004556.1,ENSG00000276345,Gene Expression,GRCh38,,,,1565.0
AC233755.1,ENSG00000275063,Gene Expression,GRCh38,,,,13.0


In [10]:
adata_small.shape

(80773, 15989)

In [11]:
rna.shape

(80773, 15972)

In [12]:
protein.shape

(80773, 17)

Normalization seems unnecessary, quality control as well since we're doing interpolation cell-by-cell, and not across different cells.

In [13]:
# 1) Split into train/test and format as Torch Tensors
# can validate the cells are the same for x, y via matching barcodes
# 2) Run via standard methods

In [14]:
gex_train = rna[:60000, :].copy()
gex_test = rna[60000:, :].copy()

adx_train = protein[:60000, :].copy()
adx_test = protein[60000:, :].copy()

In [15]:
gex_train

AnnData object with n_obs × n_vars = 60000 × 15972
    obs: 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'pattern', 'read', 'sequence', 'n_counts'
    layers: 'counts'

In [16]:
# Validate via barcodes & convert to tensors  
if (gex_train.obs.index.tolist() == adx_train.obs.index.tolist()) and (gex_test.obs.index.tolist() == adx_test.obs.index.tolist()):
    print("Looks good!") 
else:
    print("Error!")
# protein.obs.index.to_list() == rna.obs.index.to_list() 

Looks good!


In [17]:
# TODO: Convert the counts etc to PyTorch tensors
def counts_to_tensor(data: ad.AnnData):
    counts_matrix = data.layers['counts'].toarray()
    counts_tensor = torch.tensor(counts_matrix, dtype=torch.float32)
    return counts_tensor

In [18]:
x_train = counts_to_tensor(gex_train)
x_test = counts_to_tensor(gex_test)

y_train = counts_to_tensor(adx_train)
y_test = counts_to_tensor(adx_test)

In [32]:
# Create TensorDataset and DataLoader
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [21]:
# Trying some different models --> see main.py
# 1) Generic Models
# 2) Specialized methods via the Dance package (currently not working, see note below)
# https://github.com/OmicsML/dance-tutorials/blob/main/dance_tutorial.ipynb

Installed DANCE version 1.0.1-dev


In [33]:
# NOTE: Babel is no good, the team is fixing the bugs making it unusable right now, try again in a week: https://discuss.dgl.ai/t/cannot-find-dgl-c-graphbolt-library/4429/12
# import os
# os.environ["DGLBACKEND"] = "pytorch"
# from pprint import pprint
# from dance.modules.multi_modality.predict_modality.babel import BabelWrapper

In [43]:
input_size = rna.n_vars         # Number of unique rna molecules
hidden_size = 128               # Hyper-parameter
output_size = protein.n_vars    # Number of unique proteins


In [44]:
# 1) FFNN
from models import FeedforwardNN 
import torch.nn as nn
import torch.optim as optim

model = FeedforwardNN(input_size, hidden_size, output_size)

criterion = nn.MSELoss()  # Use nn.CrossEntropyLoss() for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [45]:
num_epochs = 100  # Adjust as needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


INTEL MKL ERROR: /home/dennis/miniconda3/envs/compbio/lib/python3.10/site-packages/torch/lib/../../../../libmkl_avx2.so.2: undefined symbol: mkl_sparse_optimize_csr_sorv_i8.
Intel MKL FATAL ERROR: Cannot load libmkl_avx2.so.2 or libmkl_def.so.2.


: 

In [ ]:
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
    
    print(f"Test Loss: {test_loss/len(test_loader):.4f}")
